# Tubes 2 ML - CNN

## Import Library

In [1]:
try:
    import tensorflow as tf
    print("TensorFlow version:", tf.__version__)
except ImportError:
    %pip install tensorflow

import numpy as np
from sklearn.metrics import f1_score
from tensorflow.keras import layers, models, losses, optimizers

TensorFlow version: 2.18.0


## Import CNN Dataset

In [2]:
# Load CIFAR-10 using tf.keras.datasets
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
print("Train shape:", x_train.shape, y_train.shape)
print("Test shape:", x_test.shape, y_test.shape)

# Normalize to [0, 1]
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train, y_test = y_train.flatten(), y_test.flatten()

# Split train to 40k train / 10k val
x_val, y_val = x_train[40000:], y_train[40000:]
x_train, y_train = x_train[:40000], y_train[:40000]

print("Train/Val/Test sizes:", len(x_train), len(x_val), len(x_test))

Train shape: (50000, 32, 32, 3) (50000, 1)
Test shape: (10000, 32, 32, 3) (10000, 1)
Train/Val/Test sizes: 40000 10000 10000


## Create CNN Model

In [3]:
class CNNModel:
    def __init__(self, conv_layers=2, filters=32, kernel_size=3, pooling='max'):
        self.conv_layers = conv_layers
        self.filters = filters
        self.kernel_size = kernel_size
        self.pooling = pooling
        self.model = self._build_model()

    def _build_model(self):
        model = models.Sequential()
        model.add(layers.Input(shape=(32, 32, 3)))

        for _ in range(self.conv_layers):
            model.add(layers.Conv2D(self.filters, self.kernel_size, activation='relu', padding='same'))
            if self.pooling == 'max':
                model.add(layers.MaxPooling2D())
            else:
                model.add(layers.AveragePooling2D())

        model.add(layers.Flatten())
        model.add(layers.Dense(128, activation='relu'))
        model.add(layers.Dense(10))
        return model

    def compile(self):
        self.model.compile(
            loss=losses.SparseCategoricalCrossentropy(from_logits=True),
            optimizer=optimizers.Adam(),
            metrics=['accuracy']
        )

    def train(self, x_train, y_train, x_val, y_val, epochs=2, batch_size=64):
        return self.model.fit(
            x_train, y_train,
            validation_data=(x_val, y_val),
            epochs=epochs,
            batch_size=batch_size
        )

    def evaluate(self, x_test, y_test):
        y_pred_logits = self.model.predict(x_test)
        y_pred = np.argmax(y_pred_logits, axis=1)
        f1 = f1_score(y_test, y_pred, average='macro')
        print("Test Macro F1 Score:", f1)
        return f1

    def save_weights(self, path="cnn.weights.h5"):
        self.model.save_weights(path)

    def load_weights(self, path="cnn.weights.h5"):
        self.model.load_weights(path)

## Train Model

In [4]:
class TrainCNN:
    def __init__(self, x_train, y_train, x_val, y_val, x_test, y_test):
        self.x_train = x_train
        self.y_train = y_train
        self.x_val = x_val
        self.y_val = y_val
        self.x_test = x_test
        self.y_test = y_test

    def run(self):
        model = CNNModel()
        model.compile()
        model.train(x_train, y_train, x_val, y_val)
        model.save_weights()
        model.evaluate(x_test, y_test)


if __name__ == "__main__":
  trainer = TrainCNN(x_train, y_train, x_val, y_val, x_test, y_test)
  trainer.run()


Epoch 1/2
625/625 ━━━━━━━━━━━━━━━━━━━━ 58s 89ms/step - accuracy: 0.3581 - loss: 1.7643 - val_accuracy: 0.5697 - val_loss: 1.2360
Epoch 2/2
625/625 ━━━━━━━━━━━━━━━━━━━━ 81s 88ms/step - accuracy: 0.5803 - loss: 1.1907 - val_accuracy: 0.6150 - val_loss: 1.0984
313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step
Test Macro F1 Score: 0.604847141807627
